In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets,transforms
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint_sequential
from torchvision import models
import time
import random

In [3]:
# utils.py
def save_net(fname, net):
    with h5py.File(fname, 'w') as h5f:
        for k, v in net.state_dict().items():
            h5f.create_dataset(k, data=v.cpu().numpy())
def load_net(fname, net):
    with h5py.File(fname, 'r') as h5f:
        for k, v in net.state_dict().items():        
            param = torch.from_numpy(np.asarray(h5f[k]))         
            v.copy_(param)
            
def save_checkpoint(state, is_best,task_id, filename='checkpoint.pth.tar'):
    if is_best:
        torch.save(state, './best_model/'+task_id+'model_best.pth.tar')   

In [4]:
args = {
    'gpu': '0',
    'task':'./best_model/',
    'original_lr':0.0004,
    'lr':0.0004,
    'batch_size':4,
    'momentum':0.95,
    'decay':5*1e-4,
    'start_epoch':0,
    'epochs':600,
    'steps':[-1,1,100,150],
    'scales':[1,1,1,1],
    'workers':4,
    'seed':time.time(),
    'print_freq':50,
    'pre':'./best_model/0.6699000587889476model_best.pth.tar',
    'test_freq':5
}

In [5]:
with open('./Data/male.txt') as f:
    male_names = f.readlines()

with open('./Data/female.txt') as f:
    female_names = f.readlines()

In [6]:
for i in range(len(male_names)):
    name = male_names[i][:-1]
    male_names[i] = name
    
for i in range(len(female_names)):
    name = female_names[i][:-1]
    female_names[i] = name

In [7]:
mf_names = []

for f_name in female_names:
    if f_name in male_names:
        mf_names.append(f_name)

print(len(mf_names))
# print(mf_names)

365


In [8]:
female_names = [f_name.lower() for f_name in female_names if not f_name in mf_names]
male_names = [m_name.lower() for m_name in male_names if not m_name in mf_names]

total_entries = len(female_names) + len(male_names)
max_len = len(max(male_names, key=len)) + len(max(female_names, key=len))
print("Total_entries = %d" % total_entries)
print("Max_len = %d" % max_len)

Total_entries = 7214
Max_len = 30


In [9]:
chars = set("".join(male_names) + "".join(female_names))
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))
print('Total_char = %d' % len(chars))

Total_char = 29


In [10]:
X = np.zeros((total_entries, max_len, len(chars)))
y = np.zeros((total_entries))

In [11]:
X.shape

(7214, 30, 29)

In [12]:
for i, name in enumerate(male_names):
    for t, char in enumerate(name):
        X[i, t, char2index[char]] = 1
#     y[i, 0]  = 1
    
for i, name in enumerate(female_names):
    for t, char in enumerate(name):
        X[i + len(male_names), t, char2index[char]] = 1
    y[i + len(male_names)]  = 1

In [13]:
class ListDataset():
    
    def __init__(self, root, shuffle=True,  train=False, batch_size=1, num_workers=4):
            random.shuffle(root)
            self.nSamples = len(root)
            self.line = root
            self.train = train
            self.batch_size = batch_size
            self.num_workers = num_workers
    
    def __len__(self):
        return self.nSamples
    
    def __getitem__(self, index):
        assert index <= len(self), 'index range error'
        return X[self.line[index]], y[self.line[index]] 

In [14]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTM(input_size=29, hidden_size=512, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=512, hidden_size=512, num_layers=1, batch_first=True)
        self.fc = nn.Linear(max_len, 2)
    
    def forward(self, names_in):
        out,_ = self.lstm1(names_in)
        out = F.dropout(out, p=0.2)
        out,_ = self.lstm2(out)
        out = out[:, :, -1]
        out = F.dropout(out, p=0.2)
        out = self.fc(out)
        out = F.softmax(out)
        return out

In [15]:
net = Net()
print(net)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
# print(optimizer)
loss_func = torch.nn.CrossEntropyLoss()

Net(
  (lstm1): LSTM(29, 512, batch_first=True)
  (lstm2): LSTM(512, 512, batch_first=True)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [16]:
def train(model, optimizer, epoch, train_list):
    losses = AverageMeter()
    batch_time = AverageMeter()
    
    train_loader = torch.utils.data.DataLoader(
        ListDataset(train_list,
                       shuffle=True, 
                       train=True, 
                       batch_size=1,
                       num_workers=args['workers']),
        batch_size=args['batch_size'],
        drop_last=True)
    
#     print(train_loader)
    
    for i, (names, genders) in enumerate(train_loader):
        names_in = torch.from_numpy(np.asarray(names)).type(torch.FloatTensor)
        names_in = Variable(names_in)
        genders_in = torch.from_numpy(np.asarray(genders)).type(torch.LongTensor)
        genders_in = Variable(genders_in)
        
        model.train()
        end = time.time()

        out = net(names_in)
#         print(out)
#         print(out.shape)
#         print(genders_in.shape)
#         print(genders_in)
        loss = loss_func(out, genders_in)
        losses.update(loss, names_in.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args['print_freq'] == 0:
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      .format(
                       epoch, i, len(train_loader), batch_time=batch_time, loss=losses))

In [17]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count  

In [ ]:
train_list = [i for i in range(len(X))]
for epoch in range(10):
    train(net, optimizer, epoch, train_list)
    torch.save(net.state_dict(), 'net_params.pkl')
    print('Model Saved!')

/home/jonathanwang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Epoch: [0][0/1803]	Time 0.457 (0.457)	Data 0.000 (0.000)	Loss 0.7205 (0.7205)	
Epoch: [0][50/1803]	Time 0.359 (0.428)	Data 0.000 (0.000)	Loss 1.0444 (0.6441)	
Epoch: [0][100/1803]	Time 0.501 (0.422)	Data 0.000 (0.000)	Loss 0.7997 (0.6274)	
Epoch: [0][150/1803]	Time 0.587 (0.423)	Data 0.000 (0.000)	Loss 0.3558 (0.6295)	
Epoch: [0][200/1803]	Time 0.357 (0.427)	Data 0.000 (0.000)	Loss 0.7616 (0.6316)	
Epoch: [0][250/1803]	Time 0.342 (0.427)	Data 0.000 (0.000)	Loss 0.3362 (0.6281)	
Epoch: [0][300/1803]	Time 0.342 (0.415)	Data 0.000 (0.000)	Loss 0.7621 (0.6277)	
Epoch: [0][350/1803]	Time 0.356 (0.406)	Data 0.000 (0.000)	Loss 0.5779 (0.6325)	
Epoch: [0][400/1803]	Time 0.456 (0.400)	Data 0.000 (0.000)	Loss 0.5816 (0.6365)	
Epoch: [0][450/1803]	Time 0.347 (0.403)	Data 0.000 (0.000)	Loss 0.5693 (0.6416)	
Epoch: [0][500/1803]	Time 0.373 (0.404)	Data 0.000 (0.000)	Loss 0.6073 (0.6472)	
Epoch: [0][550/1803]	Time 0.346 (0.402)	Data 0.000 (0.000)	Loss 0.4287 (0.6490)	
Epoch: [0][600/1803]	Time 0.377

In [26]:
# Apply.py
def predict(model, name, max_len, chars, char2index):
    name = name.lower()
    q_name = torch.zeros((1, max_len, len(chars)))
    for i, char in enumerate(name):
        q_name[0, i, char2index[char]] = 1
    prob = model(q_name)
    return prob

gender_predictor = Net()
gender_predictor.load_state_dict(torch.load('net_params.pkl'))

while True:
    query_name = input('Input a name: ')
    if query_name == '-1':
        break
    v = predict(gender_predictor, query_name, max_len, chars, char2index)[0]
#     print(v)
    if v[0] > v[1]:
        print('Male')
    else:
        print('Female')

Input a name: Jonathan


/home/jonathanwang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Male
Input a name: Lily
Female
Input a name: -1
